In [4]:
import numpy as np

In [5]:
def compute_relative_gradient(F_prime, z0):
    """
    Compute the relative gradient (g_i) as described in Equation 19.4.

    Parameters:
    - F_prime: np.ndarray, shape (n_samples, n_features)
        The matrix of partial derivatives of the function F with respect to the coefficients.
    - z0: np.ndarray, shape (n_samples,)
        The predicted values (z0) from the current iteration.

    Returns:
    - g_i: np.ndarray, shape (n_samples, n_features)
        The relative gradient for each sample.
    """
    # Ensure z0 does not contain zeros to avoid division by zero
    z0 = np.where(z0 == 0, 1e-8, z0)  # Replace zeros with a small value
    g_i = F_prime / z0[:, np.newaxis]  # Compute the relative gradient
    return g_i

In [3]:
from scipy.optimize import approx_fprime
import numpy as np

# Define the prediction function
def F(c, X):
    return np.dot(X, c)

# Example data
X = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])  # Design matrix
c = np.array([0.5, 1.0])  # Coefficients

# Compute numerical partial derivatives
epsilon = 1e-8  # Small step for numerical differentiation
F_prime = np.array([approx_fprime(c, lambda c: F(c, X[i, :]), epsilon) for i in range(X.shape[0])])
print("F_prime (Numerical Partial Derivatives):")
print(F_prime)

F_prime (Numerical Partial Derivatives):
[[0.99999999 2.        ]
 [3.00000006 4.        ]
 [5.00000003 6.        ]]


In [ ]:
def compute_G_i(f_e, D_prime, g_i):
    """
    Compute G_i as described in Equation 19.3, using f_e^2.

    Parameters:
    - f_e: np.ndarray, shape (n_samples,)
        The filtering weights (f_e) for each sample.
    - D_prime: np.ndarray, shape (n_samples,)
        The derivative of the function D with respect to the predicted values (z0).
    - g_i: np.ndarray, shape (n_samples, n_features)
        The relative gradient (output of compute_relative_gradient).

    Returns:
    - G_i: np.ndarray, shape (n_samples, n_features)
        The computed G_i for each sample.
    """
    # Ensure D_prime does not contain zeros to avoid division by zero
    D_prime = np.where(D_prime == 0, 1e-8, D_prime)  # Replace zeros with a small value

    # Compute G_i using f_e^2
    G_i = (f_e ** 2)[:, np.newaxis] * np.sqrt(-D_prime)[:, np.newaxis] * g_i
    return G_i

# Example usage
if __name__ == "__main__":
    # Example data
    f_e = np.array([0.8, 0.9, 1.0])  # Filtering weights
    D_prime = np.array([-0.5, -0.3, -0.1])  # Derivative of D with respect to z0 (negative values)
    g_i = np.array([[0.4, 0.8], [0.5, 0.7], [0.6, 0.9]])  # Relative gradient (output of compute_relative_gradient)

    # Compute G_i
    G_i = compute_G_i(f_e, D_prime, g_i)
    print("G_i (Equation 19.3 with f_e^2):")
    print(G_i)

In [ ]:
def compute_G_i(f_e, h_c, D_func, g_i):
    """
    Compute G_i as described in Equation 19.3, using f_e^2 and D(h_c).

    Parameters:
    - f_e: np.ndarray, shape (n_samples,)
        The filtering weights (f_e) for each sample.
    - h_c: np.ndarray, shape (n_samples,)
        The gnostic characteristic (e.g., h_i or h_j) for each sample.
    - D_func: callable
        A function that computes D(h_c), D'(h_c), and D''(h_c).
        It should return a tuple: (D(h_c), D'(h_c), D''(h_c)).
    - g_i: np.ndarray, shape (n_samples, n_features)
        The relative gradient (output of compute_relative_gradient).

    Returns:
    - G_i: np.ndarray, shape (n_samples, n_features)
        The computed G_i for each sample.
    """
    # Compute D(h_c), D'(h_c), and D''(h_c) using the provided function
    _, D_prime, _ = D_func(h_c)

    # Ensure D'(h_c) does not contain zeros to avoid division by zero
    D_prime = np.where(D_prime == 0, 1e-8, D_prime)  # Replace zeros with a small value

    # Compute G_i using f_e^2
    G_i = (f_e ** 2)[:, np.newaxis] * np.sqrt(-D_prime)[:, np.newaxis] * g_i
    return G_i

# Example D(h_c) function
def D_example(h_c):
    """
    Example implementation of D(h_c), D'(h_c), and D''(h_c).

    Parameters:
    - h_c: np.ndarray, shape (n_samples,)
        The gnostic characteristic (e.g., h_i or h_j).

    Returns:
    - D: np.ndarray, shape (n_samples,)
        The value of D(h_c).
    - D_prime: np.ndarray, shape (n_samples,)
        The first derivative of D(h_c).
    - D_double_prime: np.ndarray, shape (n_samples,)
        The second derivative of D(h_c).
    """
    # Example: D(h_c) = -log(h_c), D'(h_c) = -1/h_c, D''(h_c) = 1/h_c^2
    D = -np.log(h_c)
    D_prime = -1 / h_c
    D_double_prime = 1 / (h_c ** 2)
    return D, D_prime, D_double_prime

# Example usage
if __name__ == "__main__":
    # Example data
    f_e = np.array([0.8, 0.9, 1.0])  # Filtering weights
    h_c = np.array([0.5, 0.7, 0.9])  # Gnostic characteristic (e.g., h_i or h_j)
    g_i = np.array([[0.4, 0.8], [0.5, 0.7], [0.6, 0.9]])  # Relative gradient (output of compute_relative_gradient)

    # Compute G_i
    G_i = compute_G_i(f_e, h_c, D_example, g_i)
    print("G_i (Equation 19.3 with f_e^2 and D(h_c)):")
    print(G_i)

In [ ]:
import numpy as np

def compute_derivatives(D_matrix, h_c, axis=0):
    """
    Compute the first and second derivatives of a matrix D(h_c) numerically.

    Parameters:
    - D_matrix: np.ndarray, shape (n_samples, n_features)
        The matrix of D(h_c) values.
    - h_c: np.ndarray, shape (n_samples,) or (n_features,)
        The gnostic characteristic values (e.g., h_i or h_j).
    - axis: int
        The axis along which to compute the derivatives (0 for rows, 1 for columns).

    Returns:
    - D_prime: np.ndarray, shape (n_samples, n_features)
        The first derivative of D(h_c).
    - D_double_prime: np.ndarray, shape (n_samples, n_features)
        The second derivative of D(h_c).
    """
    # Compute the spacing (assumes uniform spacing in h_c)
    delta_h = np.gradient(h_c, axis=0 if axis == 0 else 1)

    # Compute the first derivative using finite differences
    D_prime = np.gradient(D_matrix, axis=axis) / delta_h

    # Compute the second derivative using finite differences
    D_double_prime = np.gradient(D_prime, axis=axis) / delta_h

    return D_prime, D_double_prime

# Example usage
if __name__ == "__main__":
    # Example D(h_c) matrix (e.g., -log(h_c))
    h_c = np.array([0.5, 0.7, 0.9])  # Gnostic characteristic values
    D_matrix = -np.log(h_c)  # Example D(h_c) values

    # Expand D_matrix to simulate a 2D matrix (e.g., for multiple features)
    D_matrix = np.tile(D_matrix, (3, 1))  # Repeat along rows for demonstration

    # Compute derivatives
    D_prime, D_double_prime = compute_derivatives(D_matrix, h_c, axis=1)

    print("D(h_c):")
    print(D_matrix)
    print("\nD'(h_c):")
    print(D_prime)
    print("\nD''(h_c):")
    print(D_double_prime)

In [ ]:
def compute_D_values(h_c, D_type="log"):
    """
    Compute D(h_c), D'(h_c), and D''(h_c) based on the type of D function specified.

    Parameters:
    - h_c: np.ndarray, shape (n_samples,)
        The gnostic characteristic values (e.g., h_i or h_j).
    - D_type: str
        The type of D function to use. Options include:
        - "log": D(h_c) = -log(h_c), D'(h_c) = -1/h_c, D''(h_c) = 1/h_c^2
        - "exp": D(h_c) = exp(-h_c), D'(h_c) = -exp(-h_c), D''(h_c) = exp(-h_c)
        - "quadratic": D(h_c) = h_c^2, D'(h_c) = 2*h_c, D''(h_c) = 2
        - "h2_div_2": D(h_c) = h_c^2 / 2, D'(h_c) = h_c, D''(h_c) = 1

    Returns:
    - D: np.ndarray, shape (n_samples,)
        The value of D(h_c).
    - D_prime: np.ndarray, shape (n_samples,)
        The first derivative of D(h_c).
    - D_double_prime: np.ndarray, shape (n_samples,)
        The second derivative of D(h_c).
    """
    if D_type == "log":
        # D(h_c) = -log(h_c)
        D = -np.log(h_c)
        D_prime = -1 / h_c
        D_double_prime = 1 / (h_c ** 2)
    elif D_type == "exp":
        # D(h_c) = exp(-h_c)
        D = np.exp(-h_c)
        D_prime = -np.exp(-h_c)
        D_double_prime = np.exp(-h_c)
    elif D_type == "quadratic":
        # D(h_c) = h_c^2
        D = h_c ** 2
        D_prime = 2 * h_c
        D_double_prime = 2 * np.ones_like(h_c)
    elif D_type == "h2_div_2":
        # D(h_c) = h_c^2 / 2
        D = (h_c ** 2) / 2
        D_prime = h_c
        D_double_prime = np.ones_like(h_c)
    else:
        raise ValueError(f"Unknown D_type: {D_type}. Supported types are 'log', 'exp', 'quadratic', and 'h2_div_2'.")

    return D, D_prime, D_double_prime

# Example usage
if __name__ == "__main__":
    # Example gnostic characteristic values
    h_c = np.array([0.5, 0.7, 0.9])

    # Compute D, D', and D'' for "h2_div_2" type
    D, D_prime, D_double_prime = compute_D_values(h_c, D_type="h2_div_2")
    print("D(h_c) for 'h2_div_2':", D)
    print("D'(h_c) for 'h2_div_2':", D_prime)
    print("D''(h_c) for 'h2_div_2':", D_double_prime)

In [ ]:
def compute_E_i(S, D_prime, D_double_prime):
    """
    Compute E_i as described in Equation 19.5.

    Parameters:
    - S: np.ndarray, shape (n_samples,)
        The scaling factor (e.g., filtering weights or another scalar).
    - D_prime: np.ndarray, shape (n_samples,)
        The first derivative of the function D with respect to the predicted values (z0).
    - D_double_prime: np.ndarray, shape (n_samples,)
        The second derivative of the function D with respect to the predicted values (z0).

    Returns:
    - E_i: np.ndarray, shape (n_samples,)
        The computed E_i for each sample.
    """
    # Ensure D_double_prime does not contain zeros to avoid division by zero
    D_double_prime = np.where(D_double_prime == 0, -1e-8, D_double_prime)  # Replace zeros with a small negative value

    # Compute E_i
    E_i = (S * D_prime) / np.sqrt(-D_double_prime)
    return E_i

# Example usage
if __name__ == "__main__":
    # Example data
    S = np.array([0.8, 0.9, 1.0])  # Scaling factor (e.g., filtering weights)
    D_prime = np.array([0.5, 0.7, 0.9])  # First derivative of D
    D_double_prime = np.array([-1.0, -1.5, -2.0])  # Second derivative of D (negative values)

    # Compute E_i
    E_i = compute_E_i(S, D_prime, D_double_prime)
    print("E_i (Equation 19.5):")
    print(E_i)

In [ ]:
def compute_delta_c_19_2(G_i, E_i):
    """
    Compute the update step (c^{k+1} - c^k) as described in Equation 19.2.

    Parameters:
    - G_i: np.ndarray, shape (n_samples, n_features)
        The computed G_i for each sample (output of compute_G_i).
    - E_i: np.ndarray, shape (n_samples,)
        The computed E_i for each sample (output of compute_E_i).

    Returns:
    - delta_c: np.ndarray, shape (n_features,)
        The update step (c^{k+1} - c^k).
    """
    # Ensure E_i does not contain zeros to avoid division by zero
    E_i = np.where(E_i == 0, 1e-8, E_i)  # Replace zeros with a small value

    # Compute the first term: sum of G_i G_i^T
    G_outer_sum = np.sum([np.outer(G_i[i], G_i[i]) for i in range(G_i.shape[0])], axis=0)

    # Compute the second term: sum of G_i / E_i (E_i is inverted here)
    G_weighted_sum = np.sum([G_i[i] * (1 / E_i[i]) for i in range(G_i.shape[0])], axis=0)

    # Compute delta_c using the formula
    delta_c = np.linalg.inv(G_outer_sum) @ G_weighted_sum
    return delta_c

# Example usage
if __name__ == "__main__":
    # Example data
    G_i = np.array([[0.4, 0.8], [0.5, 0.7], [0.6, 0.9]])  # Example G_i values
    E_i = np.array([0.4, 0.5, 0.6])  # Example E_i values

    # Compute delta_c
    delta_c = compute_delta_c_19_2(G_i, E_i)
    print("Delta c (c^{k+1} - c^k):")
    print(delta_c)

# good outline 

In [ ]:
import numpy as np

class RobustRegression:
    """
    A class to perform robust regression using Equations 19.1 to 19.5.
    """

    def __init__(self, X, y, max_iter=100, tol=1e-6, S=1.0, D_type="h2_div_2"):
        """
        Initialize the robust regression model.

        Parameters:
        - X: np.ndarray, shape (n_samples, n_features)
            The design matrix (input features).
        - y: np.ndarray, shape (n_samples,)
            The target values.
        - max_iter: int
            Maximum number of iterations.
        - tol: float
            Tolerance for convergence.
        - S: float
            Scaling factor for E_i (Equation 19.5).
        - D_type: str
            Type of D function to use (e.g., "h2_div_2", "log", "exp", "quadratic").
        """
        self.X = X
        self.y = y
        self.max_iter = max_iter
        self.tol = tol
        self.S = S
        self.D_type = D_type
        self.c = np.zeros(X.shape[1])  # Initialize coefficients

    def compute_relative_gradient(self, F_prime, z0):
        """
        Compute the relative gradient (g_i) as described in Equation 19.4.
        """
        z0 = np.where(z0 == 0, 1e-8, z0)  # Avoid division by zero
        g_i = F_prime / z0[:, np.newaxis]
        return g_i

    def compute_G_i(self, f_e, h_c, D_func, g_i):
        """
        Compute G_i as described in Equation 19.3.
        """
        _, D_prime, _ = D_func(h_c)
        D_prime = np.where(D_prime == 0, 1e-8, D_prime)  # Avoid division by zero
        G_i = (f_e ** 2)[:, np.newaxis] * np.sqrt(-D_prime)[:, np.newaxis] * g_i
        return G_i

    def compute_E_i(self, S, D_prime, D_double_prime):
        """
        Compute E_i as described in Equation 19.5.
        """
        D_double_prime = np.where(D_double_prime == 0, -1e-8, D_double_prime)  # Avoid division by zero
        E_i = (S * D_prime) / np.sqrt(-D_double_prime)
        return E_i

    def compute_delta_c(self, G_i, E_i):
        """
        Compute the update step (c^{k+1} - c^k) as described in Equation 19.2.
        """
        E_i = np.where(E_i == 0, 1e-8, E_i)  # Avoid division by zero
        G_outer_sum = np.sum([np.outer(G_i[i], G_i[i]) for i in range(G_i.shape[0])], axis=0)
        G_weighted_sum = np.sum([G_i[i] * (1 / E_i[i]) for i in range(G_i.shape[0])], axis=0)
        delta_c = np.linalg.inv(G_outer_sum) @ G_weighted_sum
        return delta_c

    def compute_D_values(self, h_c):
        """
        Compute D(h_c), D'(h_c), and D''(h_c) based on the specified D_type.
        """
        if self.D_type == "h2_div_2":
            D = (h_c ** 2) / 2
            D_prime = h_c
            D_double_prime = np.ones_like(h_c)
        elif self.D_type == "log":
            D = -np.log(h_c)
            D_prime = -1 / h_c
            D_double_prime = 1 / (h_c ** 2)
        elif self.D_type == "exp":
            D = np.exp(-h_c)
            D_prime = -np.exp(-h_c)
            D_double_prime = np.exp(-h_c)
        elif self.D_type == "quadratic":
            D = h_c ** 2
            D_prime = 2 * h_c
            D_double_prime = 2 * np.ones_like(h_c)
        else:
            raise ValueError(f"Unknown D_type: {self.D_type}")
        return D, D_prime, D_double_prime

    def fit(self):
        """
        Fit the robust regression model using the iterative procedure.
        """
        for iteration in range(self.max_iter):
            # Compute predicted values
            z0 = self.X @ self.c

            # Compute partial derivatives (F_prime)
            F_prime = np.array([np.gradient(z0, self.c[j]) for j in range(self.X.shape[1])]).T

            # Compute relative gradient (g_i)
            g_i = self.compute_relative_gradient(F_prime, z0)

            # Compute h_c (gnostic characteristic)
            h_c = np.abs(self.y - z0)

            # Compute D(h_c), D'(h_c), and D''(h_c)
            D, D_prime, D_double_prime = self.compute_D_values(h_c)

            # Compute filtering weights (f_e)
            f_e = np.ones_like(h_c)  # Example: uniform weights (can be customized)

            # Compute G_i
            G_i = self.compute_G_i(f_e, h_c, self.compute_D_values, g_i)

            # Compute E_i
            E_i = self.compute_E_i(self.S, D_prime, D_double_prime)

            # Compute delta_c
            delta_c = self.compute_delta_c(G_i, E_i)

            # Update coefficients
            self.c += delta_c

            # Check for convergence
            if np.linalg.norm(delta_c) < self.tol:
                print(f"Converged in {iteration + 1} iterations.")
                break

    def predict(self, X):
        """
        Predict target values for given input data.
        """
        return X @ self.c